In [ ]:
#initializing the particle positions

In [ ]:
import numpy as np

num=20
space=.6
ar=np.zeros((num*num,3))
x1=-3
d=0
for i in range(num):
         
    y1=0 
    for j in range(num):
                  
        ar[d] = [x1, y1,0] 
        y1=y1+space
        d=d+1
            
    x1=x1+space
v=tuple(ar)   

 
x_initial=np.reshape(v, (-1, 1))     
 

In [ ]:
# determining  tuple h, ecah of whose elements will  indicate a pair of the previously initialized  particles, such that when all 
#the pairs are connected with springs,it will yield the sytem of particles connected both vertically and horizontally with springs

# h will be used for determining L, j , d while solving eqn:14 as well as for connecting the particle pairs with straight line 
# after evry time step

In [ ]:

 
d=0
s=0
lst1=[None] * num*(num-1)


for i in range(num):
    
    for j in range(num):        
    
         
        if d>0:
            
            if d%num!=0:
                lst1[s] = (d,d-1) 
                s=s+1
             
        d=d+1
         
     
    
r=tuple(lst1)



d=0
s=0
ls=[None] * num*(num-1)

for i in range(num):
     
    for j in range(num):
        
        if i<(num-1):            
            ls[s] = (d,d+num) 
            s=s+1
        d=d+1     
            
r1=tuple(ls)     


 
a=np.array(r)
b=np.array(r1)
c=np.vstack((a,b))
h=tuple(map(tuple, c))

 


In [ ]:
#determining the system constants(L, J, M)

In [ ]:
spring_constant=29+12+29+8+30
A=np.zeros([num*num,num*num])
for s in range(len(h)):
    p=np.zeros(num*num)[np.newaxis].T
    p[h[s][0] ]=1 
    p[h[s][1] ]=-1
    A=A+np.dot(p,p.T)*spring_constant
    
L=np.kron(A,np.identity(3))   


A=np.zeros([num*num,len(h)])
for s in range(len(h)):
    p=np.zeros(num*num)[np.newaxis].T
    p[h[s][0] ]=1 
    p[h[s][1] ]=-1
    S=np.zeros(len(h))[np.newaxis].T
    S[s]=1
    A=A+np.dot(p,S.T)*spring_constant 
    
    
J=np.kron(A,np.identity(3))

mass=.3

M=np.zeros([num*num,num*num]) 
np.fill_diagonal(M, mass)
M=np.kron(M,np.identity(3)) 




In [ ]:
f_gravity=np.zeros((num*num,3)) 

for i in range(num*num):
    f_gravity[i]=[0,-.8,0]
    
f_gravity=np.reshape(f_gravity, (-1, 1))                    

    
    

In [ ]:
 


import pygame
from pygame.locals import *

from OpenGL.GL import *
from OpenGL.GLU import *
import random
 
 
    


def Cloth(p):
      
    glBegin(GL_LINES)
    for edge in h:
        for vertex in edge:
            glVertex3fv(p[vertex])   
    glEnd()
    

    
a=0    
r= [[[] for i in range(2)] for i in range(1000000) ] 
p=np.zeros((len(h),3))
d=np.zeros(3*len(h))[np.newaxis].T    
step_squared=.005
rest_length=space

c=np.linalg.pinv(M+L*step_squared)
 
y=x_initial
x_new=np.reshape(x_initial, (-1, 3))



pygame.init()


display = (800,600)
pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
gluPerspective(45, (display[0]/display[1]), 0.1, 50.0)
glTranslatef(0,0,-30)

 

while True:
    for event in pygame.event.get():
        
        if event.type == pygame.QUIT:
            print(a)
            pygame.quit()
            quit()
     
   
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
    
    
    
    f_random=np.random.uniform(low=-3, high=3, size=(num*num,3))    
    f_random=np.reshape(f_random, (-1, 1))
      
    
     
    r[a]=np.reshape(x_new, (-1, 1))     
        
    if a>0:
        y=2*r[a]-r[a-1]
        x_new=np.reshape(y, (-1, 3))    
    a=a+1
     
        
        
    #updatig the particle positions by solving for eqn: 14 
    for k in range(1): 
         
        for i in range(len(h)):
            
            p[i]=((x_new[h[i][0]]-x_new[h[i][1]])/(np.linalg.norm(x_new[h[i][0]]-x_new[h[i][1]])))*rest_length
            
        d=np.reshape(p, (-1, 1))    
        
        
        x_new=np.dot(c,(np.dot(M,y) + np.dot(J,d)*step_squared +(f_random+f_gravity)*step_squared)) 
        x_new=np.reshape(x_new, (-1, 3))
        x_new[num*num-1]=[8,10,0]
        x_new[num-1]=[-7,10,0]
      
    x_new=tuple(x_new) 
     
        
    Cloth(x_new)
    pygame.display.flip()
    pygame.time.wait(10)


 